<a href="https://colab.research.google.com/github/jsameijeiras/joses-notes/blob/master/Restaurants_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Análisis de cartas gallegas extraídas de El Tenedor 
---


Esto es un ejemplo de como estudiar los datos extraídos del Tenedor utilizando Python y las librería Pandas, Plotly y Folium. La intención es demostrar el poder de los datos y las facilidades actualmente para tratarlos, en cuestiones que hace 10 años necesitaría de encuestas y asisitir a todos los restaurantes aquí analizados. 

Con la intención de no expandir demasiado el análsis se ha decidido reducir el análisis a los siguientes 3 puntos:


*   Precio de una tapa de pulpo: variaciones, efecto localización.
*   Es el pescado o la carne más común en Galicia?
*   ¿Que restaurante tiene la mayor cantidad de postres?





In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive



Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [0]:
import pandas as pd
path = "/gdrive/My Drive/the_fork/thefork_menus_db.csv"

restaurants_db = pd.read_csv(path)

Antes de proceder al analísis es importante tener una idea de cual es el formato del dataset que vamos a tratar. Normalmente el ejercicio de exploración debe ser más extenso y en futuros posts intentaré dedicarle tiempo a esto porque es clave para ser lo más precisos posible. Pero dado que hemos construído nosotros mismos la tabla que vamos a usar y además contiene muy pocos datos nos basta con echarle un vistazo a la cabeza y resumir una serie de metricas. 

In [3]:
restaurants_db.head()

,restaurant_name,category,lat,long,city,name,isMainDish,description,offers.price
0,El de Alberto,Pescados,43.376595,-8.400932,A Coruña,Lomo de bacalao con muselina de ajo sobre cama...,True,NaN,18.0
1,El de Alberto,Pescados,43.376595,-8.400932,A Coruña,Merluza del pincho sobre crema de patata a la ...,True,NaN,17.0
2,El de Alberto,Pescados,43.376595,-8.400932,A Coruña,Salmonetes con deconstrucción de escabeche,True,NaN,17.5
3,El de Alberto,Pescados,43.376595,-8.400932,A Coruña,Atún rojo marinado en soja y curry,True,NaN,19.0
4,El de Alberto,Carnes,43.376595,-8.400932,A Coruña,Solomillo de ternera gallega con foie,False,NaN,18.5


Lo primero en lo que caemos es que los precios son demasiado dispares. En el siguiente grafico dibujamos un Box Plot que nos permiete identificar las medias y los cuartiles a la vez que la distribución de precios en función de la ciudad que queremos estudiar. 

Muy útil para entender que Pontevedra tiene menos restaurantes y por lo tanto un precio mediano más alto que A Coruña, el hecho de tener más variedad da mayor oportunidades y por lo tanto hace que hay dos euros de diferencia entre la ciudad más rica de de Galicia y la cuarta. Fijense que sucede algo similar con Vigo.




In [4]:
import plotly.express as px
fig = px.box(restaurants_db, x="city", y="offers.price", points="all")
fig.show()

# 1. Cuanto cuesta comer pulpo en la distintas ciudades de Galicia?


---

Para poder estudiar está pregunta lo primero que hacemos es construír una tabla donde solo estudiemos aquellos platos que contiene la palabra "Pulpo". Una vez los tenemos identificados vamos a dibujar un gráfico similar al anterior. 




In [5]:
#restaurants_db['split_words'] = restaurants_db['name'].split(' ')
pulpo = 'Pulpo|pulpo|PULPO|polbo|pulpo a la gallega|polbo a feira'
restaurants_db["contains_octopus"] = restaurants_db["name"].str.contains(pulpo)

tapas_pulpo = restaurants_db[restaurants_db['contains_octopus'] == True]
tapas_pulpo.head()

,restaurant_name,category,lat,long,city,name,isMainDish,description,offers.price,contains_octopus
69,Árbore Da Veira de Luis Veira,LOS CLÁSICOS DE ÁRBORE DA VEIRA,43.377278,-8.433006,A Coruña,"Pulpo a la brasa con alcachofas, jamón y morcilla",True,NaN,NaN,True
90,URO Rest & Club,Del mar,43.367220,-8.407043,A Coruña,Pulpo a la parrilla con aliño de pimiento chip...,False,NaN,18.0,True
166,Mamá Chicó - Picavia,Sugerencias del Chef,43.365407,-8.405758,A Coruña,Pulpo a la plancha,False,con puré de patata trufado y salsa pesto,17.9,True
239,Entre Bateas,Plato,43.370903,-8.398371,A Coruña,"Pulpo grillado, cachelos fritos y grelos salte...",True,NaN,17.5,True
340,Pablo Gallego,Entrante,43.370524,-8.394839,A Coruña,Pulpo a la gallega,True,NaN,16.0,True


In [6]:
fig = px.box(tapas_pulpo, x="city", y="offers.price", points="all")
fig.show()


In [8]:
import folium
from folium.plugins import MarkerCluster 

map = folium.Map(location=[42.7551, -7.8662])

tapas_pulpo = tapas_pulpo.reset_index()

for i in range(tapas_pulpo.shape[0]):
    location = [tapas_pulpo['lat'][i],tapas_pulpo['long'][i]]
    
    folium.Marker(location,
                  popup="""
                  <i>Precio de la tapa de pulpo: </i> <br> <b>{}Euros</b> <br> """.format(
                    round(tapas_pulpo['offers.price'][i],2))).add_to(map)
map

# 2. ¿Qué categoría es tiene más platos? ¿Las Carnes o los Pescados?


Por limitaciones en el tiempo de análisis he decidido restringir está sección a estudiar solo aquellos restaurantes que tenga una categoría de Pescados y/o Carnes. Hay suficiente platos como para poder analizar y el coste de limpiar todos los datos era superior a la intención de este análisis que solo pretende demostrar algunas de las posibilidades.

In [9]:
#carne_pescado = 'Pescados|Carnes'
#restaurants_db["c_filtered"] = restaurants_db["category"].str.contains(carne_pescado)

res_cp = restaurants_db[(restaurants_db['category'] == 'Pescados')| (restaurants_db['category'] == 'Carnes')]
res_cp.head()

,restaurant_name,category,lat,long,city,name,isMainDish,description,offers.price,contains_octopus
0,El de Alberto,Pescados,43.376595,-8.400932,A Coruña,Lomo de bacalao con muselina de ajo sobre cama...,True,NaN,18.0,False
1,El de Alberto,Pescados,43.376595,-8.400932,A Coruña,Merluza del pincho sobre crema de patata a la ...,True,NaN,17.0,False
2,El de Alberto,Pescados,43.376595,-8.400932,A Coruña,Salmonetes con deconstrucción de escabeche,True,NaN,17.5,False
3,El de Alberto,Pescados,43.376595,-8.400932,A Coruña,Atún rojo marinado en soja y curry,True,NaN,19.0,False
4,El de Alberto,Carnes,43.376595,-8.400932,A Coruña,Solomillo de ternera gallega con foie,False,NaN,18.5,False


In [10]:
fig = px.box(res_cp, x="category", y="offers.price", points="all")
fig.show()

In [11]:
res_cp.groupby('category').agg({'offers.price': ['count','median']}).sort_values(by = ('offers.price','count'), ascending = False)

offers.price       
                count median
category                    
Carnes             88   17.5
Pescados           49   19.0

# 3. Que pasa con los postres en los restaurantes gallegos?

En esta sección vemos lo fácil que es identificar aquellos restaurantes con más Postres que la media 

In [0]:
postres = restaurants_db[(restaurants_db['category'] == "Postre") | (restaurants_db['category'] == "Postres")]

In [13]:
tabla_postres = postres.groupby('restaurant_name').agg({'offers.price': ['count','median']}).sort_values(by = ('offers.price','count'), ascending = False)

print("Precio mediano de postres ofrecidos en los menus gallegos:", postres['offers.price'].median())
print("Mediana de cantidad de postres ofrecidos en los menus gallegos:", tabla_postres[('offers.price','count')].median())

tabla_postres.head(10)


Precio mediano de postres ofrecidos en los menus gallegos: 4.0
Mediana de cantidad de postres ofrecidos en los menus gallegos: 3.0


offers.price       
                                                   count median
restaurant_name                                                
Vinopremier                                           16    4.0
Masala                                                12    4.5
Pitón                                                 11    3.0
A Ponte Asador                                        11    3.0
Mamá Chicó - Picavia                                   8    5.3
Samaná Coruña                                          7    6.5
Restaurante Enxebre - Parador de Pontevedra            7    5.0
El cristal de San Roque                                7    2.7
Amares Tapas e Peixes                                  6    4.5
Adega das Caldas                                       6    5.0

# Conclusión
Como se ve en este pequeño analísis los precios de las cartas de un restaurante son muy dispares y hay muchas categorías que se pueden estudiar. Esto solo es un intento de visualizar las posibilidades que tiene la ciencia de datos en sectores menos informatizados. 

Hemos recogido las cartas de los más de 100 restaurantes que hay anunciados en el Tenedor. Teniendo acceso directo a los precios anunciados y la geolocalización de donde se ofrecen esos productos. Además contamos con las categorías y las descripciones lo cual nos permite entender mejor y poder encontrar elementos comparables a la hora de preparar nuestras propias cartas. 

Si quieres tener acceso a la BBDD o consultar más datos de este tipo o de cualquier otro sector no dudes en contactarme en: jsameijeiras@gmail.com
